In [1]:
%load_ext autoreload
%autoreload 2

In [49]:
import requests
from session import SESSION
from common import print_problem, get_problem_input, submit_answer, neighbors
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from collections import Counter, namedtuple
import re
from collections import defaultdict
from copy import copy, deepcopy
import itertools
from itertools import accumulate, zip_longest
from pprint import pprint
from functools import cache
import numpy as np
from enum import Enum, IntEnum
from string import ascii_lowercase, ascii_uppercase
from dataclasses import dataclass, field
import operator
from typing import List, Tuple, Set
import functools

In [3]:
DAY = 13

In [4]:
raw_data, data = get_problem_input(DAY)

Raw Data:
'[[9,[8,[7]],4,7,[[4,9,3,2],6,5,[1,10,2,4,5],[1,3,2,6,1]]],[4,[6],3],[4,[7,0,3,8,'
Split Data:
['[[9,[8,[7]],4,7,[[4,9,3,2],6,5,[1,10,2,4,5],[1,3,2,6,1]]],[4,[6],3],[4,[7,0,3,8,8],[7,[10],[10,10,10,7]]]]',
 '[[],[[6,4,[5],4,[0,5,5]],3,[5,[],9]],[[[5,1,6,2],[6,4,7],[4,7,4,2]],[10,[],[6,9,4],[6,2],9]],[[[10,5,9],10,0,[4,1,4,10,1],1],[7,7,[10,8],6],2]]',
 '',
 '[[[8,2],2,[[10],[],6,10,[10,10,3,10,3]]],[],[]]',
 '[[[[7,3,5],10,[4,1,0,8,7],9],10,[]]]',
 '',
 '[[6,2,[2,[10,10,0],[2]],7]]',
 '[[7,9],[5,[8,[0,5,5,10,4],6,8,6],3,[4],2],[4]]',
 '',
 '[[[5],[6,[5,3,0],[6,2,5,2,6],1,6],[[]],[3,[1,1,0],[],1,[9,4,3,5,7]]],[[1,9,[2,2,6],[8,9],2],1,[[],0,8],[[],9],3]]']


In [17]:
test_data = [
"[1,1,3,1,1]",
"[1,1,5,1,1]",
"",
"[[1],[2,3,4]]",
"[[1],4]",
"",
"[9]",
"[[8,7,6]]",
"",
"[[4,4],4,4]",
"[[4,4],4,4,4]",
"",
"[7,7,7,7]",
"[7,7,7]",
"",
"[]",
"[3]",
"",
"[[[]]]",
"[[]]",
"",
"[1,[2,[3,[4,[5,6,7]]]],8,9]",
"[1,[2,[3,[4,[5,6,0]]]],8,9]",
]

In [6]:
def parse(str_data):
    ret = []
    str_iter = iter(str_data)
    try:
        while True:
            a = next(str_iter)
            b = next(str_iter)
            ret.append((eval(a), eval(b)))
            _ = next(str_iter)
    except StopIteration:
        pass
        
    return ret

In [31]:
def compare(a: List | int, b: List | int, level=0, p=False):
    if p: print((" "*level*2) + f"- Compare {str(a).replace(' ','')} vs {str(b).replace(' ','')}")
    if isinstance(a, int) and isinstance(b, int):
        if a < b:
            if p: print((" "*(level+1)*2) + "- Left side is smaller, so inputs are in the right order")
            return True
        elif a == b:
            return None
        else:
            if p: print((" "*(level+1)*2) + "- Right side is smaller, so inputs are not in the right order")
            return False
            
    
    elif isinstance(a, list) and isinstance(b, list):
        for _a, _b in zip_longest(a,b,fillvalue=None):
            if _a is None:
                if p: print((" "*(level+1)*2) + "- Left side ran out of items, so inputs are in the right order")
                return True
            elif _b is None:
                if p: print((" "*(level+1)*2) + "- Right side ran out of items, so inputs are not in the right order")
                return False
            
            subcompare = compare(_a,_b, level+1,p=p)
            
            if subcompare is not None:
                return subcompare
        
        return None
    
    else: #mixed int and list
        if isinstance(a, int) and isinstance(b, list):
            if p: print((" "*(level+1)*2) + f"- Mixed types; convert left to {[a]} and retry comparison")
            return compare([a],b, level+1,p=p)
        if isinstance(a, list) and isinstance(b, int):
            if p: print((" "*(level+1)*2) + f"- Mixed types; convert right to {[b]} and retry comparison")
            return compare(a,[b], level+1,p=p)
        else:
            raise ValueError

In [41]:
def compare(a: List | int, b: List | int, level=0, p=False):
    if p: print((" "*level*2) + f"- Compare {str(a).replace(' ','')} vs {str(b).replace(' ','')}")
    
    if isinstance(a, int) and isinstance(b, int):
        if a < b:
            if p: print((" "*(level+1)*2) + "- Left side is smaller, so inputs are in the right order")
            return 1
        elif a == b:
            return 0
        else:
            if p: print((" "*(level+1)*2) + "- Right side is smaller, so inputs are not in the right order")
            return -1
            
    
    elif isinstance(a, list) and isinstance(b, list):
        for _a, _b in zip_longest(a,b,fillvalue=None):
            if _a is None:
                if p: print((" "*(level+1)*2) + "- Left side ran out of items, so inputs are in the right order")
                return 1
            elif _b is None:
                if p: print((" "*(level+1)*2) + "- Right side ran out of items, so inputs are not in the right order")
                return -1
            
            subcompare = compare(_a,_b, level+1,p=p)
            
            if subcompare != 0:
                return subcompare
        
        return 0
    
    else: #mixed int and list
        if isinstance(a, int) and isinstance(b, list):
            if p: print((" "*(level+1)*2) + f"- Mixed types; convert left to {[a]} and retry comparison")
            return compare([a],b, level+1,p=p)
        if isinstance(a, list) and isinstance(b, int):
            if p: print((" "*(level+1)*2) + f"- Mixed types; convert right to {[b]} and retry comparison")
            return compare(a,[b], level+1,p=p)

In [42]:
def find_indices(str_data, p=False):
    indices = set()
    for i, x in enumerate(parse(str_data), start=1):
        if p: print(f"== Pair {i} ==")
        if compare(*x, p=p) > 0:
            indices.add(i)

        if p: print("")
    return sum(indices)

In [44]:
find_indices(data,False)

6272

In [13]:
submit_answer(DAY, 1, Out[max(Out.keys())])

6272
("That's the right answer!  You are one gold star closer to collecting enough "
 'star fruit. [Continue to Part Two]')


True

In [70]:
def sort_items(str_data, p=False):
    num_data = parse(str_data)
    
    unrolled = []
    for a,b in num_data:
        unrolled.append(a)
        unrolled.append(b)
    
    x = eval("[[2]]")
    y = eval("[[6]]")
    
    unrolled.append(x)
    unrolled.append(y)
    
    unrolled.sort(key=functools.cmp_to_key(compare), reverse=True)
    
    
    result = 1
    
    for i, u in enumerate(unrolled, start=1):
        if u == x:
            result *= i
        if u == y:
            result *= i

    return result

In [72]:
sort_items(data)

22288

In [73]:
submit_answer(DAY, 2, Out[max(Out.keys())])

22288
("That's the right answer!  You are one gold star closer to collecting enough "
 'star fruit.You have completed Day 13! You can [Shareon\n'
 '  Twitter\n'
 'Mastodon] this victory or [Return to Your Advent Calendar].')


True